# Assignment 1 - Exercise Setup
This notebook contains the necessary code setup for the accompanying exercises. 

# 1. Coordinate Descent

In [2]:
import numpy as np
def argmin_x1(x):
      return x[2] + (3/2)*x[1] - 0.5

def argmin_x2(x):
    return (1/6)*(x[0] + 2*x[2] + 5)

def argmin_x3(x):
    return (1/4)*(x[0] + 3*x[1] - 4)

def f(x):
    return np.exp(x[0] - 3*x[1] + 3) + \
           np.exp(3*x[1] - 2*x[2] - 2) + \
           np.exp(2*x[2] - x[0] + 2)

def coordinate_descent(f, argmin, x0, max_iter=100, verbose=False):
    x = np.array(x0, dtype=float)
    # to track the trajectory of x
    history = [x.copy()]

    for t in range(max_iter):
        for i in range(len(x)):
             # in-place update
            x[i] = argmin[i](x)
        history.append(x.copy())

    return x

# 2. Gradient Descent

In [3]:
def f(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

def gradient_descent(f, grad_f, eta, u0, v0, max_iter=100) -> tuple[list, list]:
    x0 = [u0, v0]
    x = np.array(x0, dtype=float)
    path = [x0]
    values = [f(x[0], x[1])]
    for t in range(1, max_iter):
        eta_t = eta(t)
        g = grad_f(x[0], x[1])
        x = x - eta_t * g
        path.append(x)
        values.append(f(x[0], x[1]))
    return values, path

def eta_const(t,c=1e-3) -> float:
    return lambda t: c

def eta_sqrt(t,c=1e-3) -> float:
    return  lambda t:( c / np.sqrt(t + 1) )

def eta_multistep(t, milestones=[30, 80, 100], c=1e-3, eta_init=1e-3) -> float:
    """
    Multi-step: each time t passes a milestone, multiply eta_init by c.
    E.g., if milestones=[30,80], c=0.1, eta_init=1e-3:
      t<30: eta=1e-3
      30<=t<80: eta=1e-4
      80<=t: eta=1e-5
    """
    drops = sum(t >= m for m in milestones)
    return eta_init * (c ** drops)

# 3. Polynomial Regression

In [6]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

# Load the dataset
housing = fetch_california_housing(as_frame=True)

# The data is in a pandas DataFrame
df = housing.frame

# View the first few rows
df.head()
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [7]:
# Splitting the dataset
from sklearn.model_selection import train_test_split

# Let's separate features and target for clarity
features = df.drop(columns=["MedHouseVal"])
target = df["MedHouseVal"]

# Standardize the features
# Your Code Here
scaled_features = ...

# Split the dataset (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(scaled_features, target, test_size=0.2, random_state=42)

TypeError: Input should have at least 1 dimension i.e. satisfy `len(x.shape) > 0`, got scalar `array(Ellipsis, dtype=object)` instead.

# 4. Bias and Variance
You can calculate the results manually, or use whatever code you would like.

In [19]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def f_1(x):
    return 2*x+4

def f_2(x):
    return x+0.1

def f_3(x):
    return 3*x+0.7

x_0=0
f_values = [f_1(0), f_2(0), f_3(0)]
sample_mean = np.mean(f_values)
# print(f"{sample_mean=}")
bias = abs(sigmoid(x_0) - sample_mean)
print(f"{bias**2=}")
variance = np.mean([(sample_mean -f_d)**2 for f_d in f_values ])
print(f"{variance=}")



bias**2=1.2099999999999997
variance=2.940000000000001


# 5. Naive Bayes

In [13]:
from sklearn.datasets import fetch_20newsgroups
categories = ['sci.space', 'misc.forsale', 'comp.graphics', 'rec.sport.hockey']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

print(train.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

=================   ==========
Classes                     20
Samples total            18846
Dimensionality               1

# 6. Decision Trees

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
print(df)
print(iris.DESCR)

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0         0  

# 7. Support Vector Machines

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split


# Set random seed for reproducibility
seed = 42
np.random.seed(seed)

# Load digits dataset
digits = datasets.load_digits()
X, y = digits.data, digits.target

# Train-test split (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 1797
:Number of Attributes: 64
:Attribute Information: 8x8 image of integer pixels in the range 0..16.
:Missing Attribute Values: None
:Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
:Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an in

In [28]:
from sklearn.svm import SVC
import sklearn

svm = SVC(gamma=0.0012, C=0.85 )
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
print(f"{accuracy=}")


accuracy=0.9907407407407407
